In [ ]:
# !pip install --upgrade pandas_datareader
# !pip install --upgrade numpy

In [ ]:
import math
import pandas as pd
import pandas_datareader as web
import numpy as np
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers, models
from tensorflow.keras.layers import *
from tensorflow.keras import backend as K
from tensorflow.keras.models import *
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from datetime import datetime

In [ ]:
seq_len = 10

In [ ]:
companies_idx = [ 'AAPL', 'MSFT', 'AMZN', 'GOOG', 'GOOGL', 'FB', 'TSLA', 'V', 'JPM', 'JNJ', 'WMT', 'NVDA', 'PYPL', 'MA', 'DIS', 'PG', 'UNH', 'HD', 'BAC', 'INTC', 'NFLX', 'CMCSA', 'ADBE', 'CRM', 'ABT', 'VZ', 'NKE', 'XOM', 'KO', 'T', 'AVGO', 'TMO', 'LLY', 'CSCO', 'PFE', 'MRK', 'PEP', 'ABBV', 'ORCL', 'CVX', 'DHR', 'ACN', 'QCOM', 'TXN', 'MDT', 'MCD', 'NEE', 'COST', 'TMUS', 'WFC', 'UNP', 'HON', 'UPS', 'MS', 'AMGN', 'PM', 'C', 'BMY', 'LIN', 'LOW', 'BA', 'SBUX', 'CHTR', 'INTU', 'NOW', 'SCHW', 'BLK', 'AMD', 'RTX', 'CAT', 'AMAT', 'GS', 'EL', 'IBM', 'AXP', 'GE', 'MMM', 'AMT', 'DE', 'MU', 'TGT', 'LMT', 'SYK', 'ISRG', 'CVS', 'BKNG', 'LRCX', 'FIS', 'SPGI', 'GILD', 'TJX', 'MO', 'ATVI', 'ZTS', 'PLD', 'MDLZ', 'GM', 'TFC', 'BDX']
no_companies = len(companies_idx)
no_companies_train = math.ceil(no_companies*0.8)

In [ ]:
list_companies = []
test = []
x = np.array([[]])
first_date = datetime.now()
time_test = []
time_train = []

In [ ]:
i = 1
for idx in companies_idx:
  stock_df = web.DataReader(idx, data_source='yahoo', start='1980-01-01', end='2021-05-20')
  stock = stock_df['Close']
  first_date = min(first_date, stock.index[0])
  stock_np = np.array(stock)
  stock_np = np.reshape(stock_np, (len(stock_np), 1))
  if i<=7:
    test.append(stock_np)
    time_test.append(stock.index)
  else:
    list_companies.append(stock_np)
    time_train.append(stock.index)
  i+=1

RemoteDataError: ignored

In [ ]:
for i in range(no_companies):
  if i>=7:
    time_train[i-7]=time_train[i-7]-first_date
    time_train[i-7] = time_train[i-7].days
    time_train[i-7] = np.array(time_train[i-7])
  else:
    time_test[i] = time_test[i]-first_date
    time_test[i] = time_test[i].days
    time_test[i] = np.array(time_test[i])

In [ ]:
# x = np.reshape(x, (len(x),1))

In [ ]:
Xtrain = []
Ytrain = []
Xtrain_time = []
br = 0
for stck, day in zip(list_companies,time_train):
  for i in range(seq_len,len(stck)):
    Xtrain_time.append(day[i-seq_len:i])
    Xtrain.append((stck[i-seq_len:i]-stck[i-1])/stck[i-1])
    Ytrain.append((stck[i]-stck[i-1])/stck[i-1])
  br+=1


In [ ]:
Xtrain_time = np.array(Xtrain_time)
Xtrain_time = np.reshape(Xtrain_time, (Xtrain_time.shape[0], Xtrain_time.shape[1], 1))

Xtrain = np.array(Xtrain)
Ytrain = np.array(Ytrain)

In [ ]:
print(Xtrain.shape)

In [ ]:
#CNN MODEL

# model = models.Sequential()

# model.add(layers.Conv1D(64, 3, activation='relu', input_shape=(Xtrain.shape[1], 1)))
# model.add(layers.MaxPooling1D(pool_size=2))

# model.add(layers.Flatten())


# model.add(layers.Dense(32))
# model.add(layers.Dense(1))

# print(model.summary())

In [ ]:
#LSTM MODEL

# model = models.Sequential()

# model.add(layers.LSTM(20, return_sequences=True, input_shape=(Xtrain.shape[1], 1)))
# model.add(layers.LSTM(20))

# model.add(layers.Dense(32))
# model.add(layers.Dense(1))

# print(model.summary())

In [ ]:
#CNN+LSTM MODEL

# model = models.Sequential()

# units = 20

# model.add(layers.Conv1D(32, 3, activation='relu', input_shape=(Xtrain.shape[1],1)))
# model.add(layers.MaxPooling1D(pool_size=2))


# model.add(tf.keras.layers.Reshape((model.output_shape[1]*model.output_shape[2],1)))

# print(model.output_shape)

# model.add(layers.LSTM(units, return_sequences=False, input_shape= (model.output_shape[1], 1)))
# #model.add(LSTM(units, return_sequences=True))


# model.add(layers.Dense(32))
# model.add(layers.Dense(1))

# print(model.summary())

In [ ]:
class Time2Vec(Layer):
    def __init__(self, vec_size):
        super(Time2Vec, self).__init__()
        self.vec_size = vec_size 
    
    def build(self, input_shape):
        self.w_linear = self.add_weight(
            shape=(1, 1),
            initializer='uniform',
            trainable=True
        )
        
        self.b_linear = self.add_weight(
            shape=(1, 1),
            initializer='uniform',
            trainable=True
        )

        self.w_periodic = self.add_weight(
            shape=(1, self.vec_size),
            initializer='uniform',
            trainable=True
        )
        
        self.b_periodic = self.add_weight(
            shape=(1, self.vec_size),
            initializer='uniform',
            trainable=True
        )
        
        super(Time2Vec, self).build(input_shape)
    
    def call(self, inputs, **kwargs):
        linear = self.w_linear * inputs + self.b_linear
        periodic = K.sin(K.dot(inputs, self.w_periodic) + self.b_periodic)
        return K.concatenate([linear, periodic], -1)
    
    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[1], self.vec_size + 1)

def time2vec_lstm(dim, t2v_dim):
  term = Input(shape=(dim, 1))
  time = Input(shape=(dim, 1))
  xti = Time2Vec(t2v_dim)(time)
  xte1 = LSTM(10, return_sequences=True)(term) 
  xte2 = LSTM(10)(xte1)
  x = Dense(1)(concatenate([Flatten()(xte2), Flatten()(xti)]))
  m = Model(inputs=[time, term],outputs=x)
  return m

In [ ]:
model = time2vec_lstm(seq_len, 16)

In [ ]:
opt = Adam(learning_rate=0.0001)
model.compile(optimizer=opt, loss='mse')

In [ ]:
#LSTM+Time2Vec fit 
history_data = model.fit(x = [Xtrain_time, Xtrain], y = Ytrain, batch_size=50, epochs=20, verbose=2, validation_split=0.2)

In [ ]:
#history_data = model.fit(Xtrain, Ytrain, batch_size=50, epochs=10, verbose=2, validation_split=0.2)

In [ ]:
plt.figure(figsize=(20,10))
plt.title('Gubitak kroz epohe')
plt.plot(history_data.history['loss'])
plt.plot(history_data.history['val_loss'])
plt.ylabel('Gubitak')
plt.xlabel('Epohe')
plt.legend(['trening gubitak' , 'validacioni gubitak'], loc = 'upper right')
plt.show()

In [ ]:
rmse = []
rmse_y = []

In [ ]:
for stck, day in zip(test,time_test):
  Xtest = []
  Ytest = []
  Xtest_time = []
  predictions_y = []
  for i in range(seq_len, len(stck)):
    Xtest.append((stck[i-seq_len:i]-stck[i-1])/stck[i-1])
    predictions_y.append(0)
    Ytest.append((stck[i]-stck[i-1])/stck[i-1])
    Xtest_time.append(day[i-seq_len:i])
  Xtest = np.array(Xtest)
  Ytest = np.array(Ytest)
  Xtest_time = np.array(Xtest_time)
  Xtest_time = np.reshape(Xtest_time, (Xtest_time.shape[0],Xtest_time.shape[1], 1))


  predictions_y = np.array(predictions_y)
  #predictions = model.predict(Xtest)
  predictions = model.predict([Xtest_time,Xtest])

  # for i in range(len(predictions)):
  #   predictions[i] = predictions[i]*stck[i-1+seq_len]+stck[i-1+seq_len]
  
  rmse.append(np.sqrt(np.mean(((predictions-Ytest)**2))))
  rmse_y.append(np.sqrt(np.mean(((predictions_y-Ytest)**2))))

In [ ]:
print(rmse)
print(rmse_y)